In [24]:
import cv2
import numpy as np
from glob import glob
import os
from sklearn.svm import LinearSVC
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from PIL import Image
import PIL.ImageOps
from glob import glob
from random import shuffle, seed
import pylab as pl
import pandas as pd
import re
from sklearn.decomposition import PCA

In [2]:
def img_to_array(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = list(img.getdata())
    img = map(list, img)
    img = np.array(img)
    s = img.shape[0] * img.shape[1]
    img_wide = img.reshape(1, s)
    return img_wide[0]

shirt = glob('bag/dataset/shirt_resized/*')
collared_tees = glob('bag/dataset/collared_tees_resized/*')
#normal_tees = glob('bag/dataset/normal_tees_resized/*')

process_file = img_to_array

raw_data = [(process_file(filename),'shirt',filename) for filename in shirt] + \
           [(process_file(filename),'collared_tees',filename) for filename in collared_tees] 
           

# randomly order the data
seed(0)
shuffle(raw_data)

In [99]:
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])
y = [0 if label == 'collared_tees' else 1 for label in labels ]

# Scaling the words
stdSlr = StandardScaler().fit(data)
data = stdSlr.transform(data)

from sklearn.cross_validation import train_test_split,cross_val_score,KFold,StratifiedShuffleSplit,StratifiedKFold
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=175)

In [100]:
pca = PCA(n_components =55, whiten=True)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

from sklearn import svm
clf = svm.SVC(kernel ='linear')
clf.fit(X_train, y_train)
# STEP 3: make predictions on the testing set
y_pred = clf.predict(X_test)

# compare actual response values (y_test) with predicted response values (y_pred)
print metrics.accuracy_score(y_test, y_pred)

0.618421052632
